Andrey_Bondarchuk_stepik_id_346032105

collab

https://drive.google.com/file/d/1RP6v4HFkzRewRU-HCKgGZYIDL2vxa4YT/view?usp=sharing

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутбука напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
#!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

описание gensim
https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
import gensim as gn
gn.__version__

'4.1.2'

In [ ]:
# размерность эмбединга
wv_embeddings.vectors.shape

(1787145, 200)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
#print(f"Num of words: {len(wv_embeddings.index2word)}")
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

используем туториал gensim

https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py

In [ ]:
# method most_simmilar
'''your code'''

# наиболее близкие к слову 'dog'
print(wv_embeddings.most_similar(positive=['dog'], topn=5))

[('animal', 0.8564180135726929), ('dogs', 0.7880866527557373), ('mammal', 0.7623804211616516), ('cats', 0.7621253728866577), ('animals', 0.760793924331665)]


In [ ]:
# наиболее близкие к слову 'cat'
print(wv_embeddings.most_similar(positive=['cat'], topn=5))

[('-vet', 0.7583354711532593), ('dog', 0.6852341890335083), ('/proc/version', 0.6815888285636902), ('feline', 0.6779966950416565), ('nyan', 0.6685197353363037)]


In [ ]:
# наиболее близкие к слову 'dogs'
print(wv_embeddings.most_similar(positive=['dogs'], topn=5))

[('cats', 0.9054571390151978), ('animals', 0.8579520583152771), ('pets', 0.796572208404541), ('dog', 0.7880866527557373), ('mammals', 0.7740597724914551)]


In [ ]:
# косинусная близость между 'dog' и 'cat'
wv_embeddings.similarity('dog', 'cat')

0.6852341

**Ответ:** 
- Слово 'cat' не входит в топ-5 близких слов для слова 'dog'. Косинусная близость данных слов = 0.685.
- Однако, cлово 'cats' входит в топ-5 для 'dog' и располагается на 4 месте.
- Для слова '**cats**' слово '**dogs**' расположено в тол-5 на **первом** месте. Косинусная близость слов = 0.905.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [ ]:
# пример работы tokenizer
tokenizer.tokenize('I love neural networks!!! Are you? Some simbols for test -- *** @@@@ --- &&&')

['I',
 'love',
 'neural',
 'networks',
 'Are',
 'you',
 'Some',
 'simbols',
 'for',
 'test']

In [ ]:
import numpy as np

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''

    embedding_dim = embeddings.vectors.shape[1]
    features = np.zeros([embedding_dim], dtype='float32')
    tokens = tokenizer.tokenize(question)
    N=0
    for word in tokens:
        if f'{word}' in embeddings.key_to_index:
            features += embeddings[f'{word}']
            N+=1 
    if N==0:
        return features.reshape(1, -1)
    else:
        features = features/N 
    return features.reshape(1, -1)

In [ ]:
question  = 'I love neural networks'
vector = question_to_vec(question, wv_embeddings, tokenizer)

In [ ]:
vector.shape

(1, 200)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
'''your code'''

question  = 'I love neural networks'
vector = question_to_vec(question, wv_embeddings, tokenizer)

third_component = round(vector[0][2],2)

print('Третья(с индексом 2) компонента вектора предложения  "I love neural networks" = {:.2f}'.format(third_component))

Третья(с индексом 2) компонента вектора предложения  "I love neural networks" = -1.29


**Ответ:** Третья(с индексом 2) компонента вектора предложения  "I love neural networks" = -1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

**Ответ:** Значение Hits@47 максимально может быть равно 1. Значение DCG@1 при большом R будет стремиться к 0. Следовательно, максимум `Hits@47 - DCG@1` будет равен 1.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

**вычислим:**

для $rank\_q_i^{'} = 2$ 

- [K = 47] $\text{Hits@47} =  [rank\_q_i^{'} \le 47] = 1$
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$



- Максимум Hits@47 - DCG@1 = 1

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
from math import log2
1/log2(10)

0.3010299956639812

**Ответ:**

- [K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = 0.3$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    hits_value = 0
    N=len(dup_ranks)
    if N==0:
        return 0
    for item in dup_ranks:
        hits_value += (item <= k)  
    hits_value = hits_value/N
    return hits_value    

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dcg_value = 0
    N=len(dup_ranks)
    if N==0:
        return 0
    for item in dup_ranks:
        dcg_value += (item <= k) / log2(item+1) 
    dcg_value = dcg_value/N

    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2] #['''your code''']

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


**Вывод:** Функции работают корректно.

### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
# только для collab
collab = False

import os
if collab and not 'stackoverflow_similar_questions' in os.listdir('./'):
    from google.colab import drive
    drive.mount('/content/gdrive/')
    !unzip -q /content/gdrive/My\ Drive/stackoverflow_similar_questions.zip

In [ ]:
#!unzip stackoverflow_similar_questions.zip

Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

In [ ]:
# посмотрим
validation_data[13][1:10]

['CSS animation not working correctly in Safari browser',
 'How to create autoresizing NSTextView for printing?',
 'Memory leak in the Win64 Delphi RTL during thread shutdown?',
 'Create an image from JSON file?',
 'Execution error: org.infinispan.util.concurrent.TimeoutException: ISPN000299: Unable to acquire lock after 10 seconds for key',
 'C - Error with function parameters when it is being called',
 'Get abs path from the currently edited file in Eclipse',
 'How to ignore return value different then 0 when I have set -e?',
 'Incremental update in sqoop']

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


In [ ]:
# посмотрим на train
train_data = read_corpus('./data/train.tsv')

train_data[0]

['converting string to list',
 'Convert Google results object (pure js) to Python object']

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''

    dict_index_cosine_similarity = {}
    vector_question = question_to_vec(question, embeddings, tokenizer)
    for index, line in enumerate(candidates):
        vector_line = question_to_vec(line, embeddings, tokenizer)
        cs = cosine_similarity(vector_question, vector_line)[0][0]
        dict_index_cosine_similarity[index] = cs
    
    cs_result = list(dict_index_cosine_similarity.items())
    cs_result.sort(key=lambda x: x[1], reverse=True)
    #print(cs_result)
    result = []
    for item in cs_result:
        result.append((item[0], candidates[0]))    
    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
    print(ranks)
    print()

[(1, 'Convert Google results object (pure js) to Python object'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'Convert Google results object (pure js) to Python object')]

[(1, 'Getting all list items of an unordered list in PHP'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'Getting all list items of an unordered list in PHP')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(1, 'Getting all list items of an unordered list in PHP'), #скрыт
            (0, 'select2 not displaying search results'), #скрыт
            (2, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ:** Для эксперимента 2 получена последовательность индексов 1,0,2

#### Оценка качества метода

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

##### Простой токенайзер

``` python
напомним

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()
```

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
%%time

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    #break

  0%|          | 0/3760 [00:00<?, ?it/s]

Wall time: 5min 31s


Результаты расчетов будем собирать в список для загрузки в pandas dataframe.

In [ ]:
result_list = []

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    # ------------------------------
    DCG = f'DCG@{k:4d}'
    Hits = f'Hits@{k:4d}'
    result_list.append({'model':'1','embedding ':'word2vec SO_vectors_200.bin','tokenizer':'simple',\
                        DCG:round(dcg_score(wv_ranking, k),3),Hits:round(hits_count(wv_ranking, k),3)})

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000


##### Токенайзер nltk + stopwords

In [ ]:
import nltk
import string

nltk.download('stopwords')
from nltk.corpus import stopwords

#stopWords = set(stopwords.words('english'))
#print(stopWords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
class NormalTokenizer:
    def __init__(self):
        self.nltk = nltk.WordPunctTokenizer()
        self.stopWords = set(stopwords.words('english'))
    def tokenize(self, text):
        tokens = self.nltk.tokenize(text.lower())    
        tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        tokens = [word for word in tokens if word not in self.stopWords]
        return tokens

normal_tokenizer = NormalTokenizer()

In [ ]:
%%time

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, normal_tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    #break

  0%|          | 0/3760 [00:00<?, ?it/s]

Wall time: 5min 54s


In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    # ------------------------------
    DCG = f'DCG@{k:4d}'
    Hits = f'Hits@{k:4d}'
    result_list.append({'model':'2','embedding ':'word2vec SO_vectors_200.bin','tokenizer':'nltk + stopwords',\
                        DCG:round(dcg_score(wv_ranking, k),3),Hits:round(hits_count(wv_ranking, k),3)})

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.395 | Hits@   1: 0.395
DCG@   5: 0.474 | Hits@   5: 0.545
DCG@  10: 0.496 | Hits@  10: 0.614
DCG@ 100: 0.544 | Hits@ 100: 0.844
DCG@ 500: 0.560 | Hits@ 500: 0.969
DCG@1000: 0.564 | Hits@1000: 1.000


**Вывод:** На улучшенном токенайзере уже виден прирост качества для загруженных эмбеддингов из "SO_vectors_200.bin".

### Эмбеддинги, обученные на корпусе похожих вопросов

**справка** пример работы с gensim

https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/

https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py

```python
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api
# Download dataset
dataset = api.load("text8")
data = [d for d in dataset]
# Split the data into 2 parts. Part 2 will be used later to update the model
data_part1 = data[:1000]

# Train Word2Vec model. Defaults result vector size = 100
model = Word2Vec(data_part1, min_count = 0, workers=cpu_count())

# Get the word vector for given word
model['topic']
#> array([ 0.0512,  0.2555,  0.9393, ... ,-0.5669,  0.6737], dtype=float32)
model.most_similar('topic')

# Save and Load Model
model.save('newmodel')
model = Word2Vec.load('newmodel')
```

##### Простой токенайзер

In [ ]:
train_data = read_corpus('./data/train.tsv')

In [ ]:
train_data[0]

['converting string to list',
 'Convert Google results object (pure js) to Python object']

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
'''your code'''
words = []
sentense_len = []
for line in train_data:
    glue = "".join(line[0]+' '+line[1])
    tokens = tokenizer.tokenize(glue)
    sentense_len.append(len(tokens)) 
    words.append(tokens) 

In [ ]:
# средняя длина предложения
np.mean(sentense_len)/2

8.728873

In [ ]:
# пример предложения
print(words[0])

['converting', 'string', 'to', 'list', 'Convert', 'Google', 'results', 'object', 'pure', 'js', 'to', 'Python', 'object']


Средняя длина предложения равна около 8 токенов. Для начала выберем размер окна по умолчанию = 5.

Выбор размера окна https://habr.com/ru/post/446530/

*Различные размеры окна подходят для разных задач. Замечено, что меньшие размеры окон (2−15) порождают взаимозаменяемые вложения с похожими индексами (обратите внимание, что антонимы часто взаимозаменяемы, если смотреть на окружающие слова: например, слова «хорошо» и «плохо» часто упоминаются в схожих контекстах). Большие размеры окон (15−50 или даже больше) порождают родственные вложения со схожими индексами. На практике вам часто придётся предоставлять аннотации ради полезного смыслового сходства в вашей задаче. В Gensim размер окна по умолчанию равен 5 (по два слова слева и справа, в дополнение к самому входному слову).*

In [ ]:
from gensim.models import Word2Vec

In [ ]:
%%time
embeddings_trained = Word2Vec(words, vector_size=200, min_count=5, window=5).wv

Wall time: 1min 56s


In [ ]:
%%time
wv_ranking = []
max_validation_examples = 1000 #1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

Wall time: 5min 47s


In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    DCG = f'DCG@{k:4d}'
    Hits = f'Hits@{k:4d}'
    result_list.append({'model':'3','embedding ':'trained','tokenizer':'simple',\
                        DCG:round(dcg_score(wv_ranking, k),3),Hits:round(hits_count(wv_ranking, k),3)})

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.261 | Hits@   1: 0.261
DCG@   5: 0.324 | Hits@   5: 0.382
DCG@  10: 0.349 | Hits@  10: 0.459
DCG@ 100: 0.402 | Hits@ 100: 0.718
DCG@ 500: 0.428 | Hits@ 500: 0.919
DCG@1000: 0.436 | Hits@1000: 1.000


##### Токенайзер nltk + stopwords

In [ ]:
train_data = read_corpus('./data/train.tsv')

words = []
sentense_len = []
for line in train_data:
    glue = "".join(line[0]+' '+line[1])
    tokens = normal_tokenizer.tokenize(glue)
    sentense_len.append(len(tokens)) 
    words.append(tokens) 

In [ ]:
%%time
embeddings_trained = Word2Vec(words, vector_size=200, min_count=5, window=5).wv

Wall time: 1min 8s


In [ ]:
%%time
wv_ranking = []
max_validation_examples = 1000 #1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, normal_tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

Wall time: 5min 49s


In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    DCG = f'DCG@{k:4d}'
    Hits = f'Hits@{k:4d}'
    result_list.append({'model':'4','embedding ':'trained','tokenizer':' nltk + stopwords',\
                        DCG:round(dcg_score(wv_ranking, k),3),Hits:round(hits_count(wv_ranking, k),3)})

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.374 | Hits@   1: 0.374
DCG@   5: 0.472 | Hits@   5: 0.555
DCG@  10: 0.494 | Hits@  10: 0.624
DCG@ 100: 0.542 | Hits@ 100: 0.859
DCG@ 500: 0.557 | Hits@ 500: 0.970
DCG@1000: 0.560 | Hits@1000: 1.000


Соберем результаты в pandas dataframe.

In [ ]:
import pandas as pd
pd_result = pd.DataFrame(result_list)

In [ ]:
pd_result = pd_result.fillna(0)

In [ ]:
pd1 = pd_result[pd_result['model']=='1'].groupby(['model','tokenizer','embedding '])[pd_result.columns[3:]].sum().reset_index()
pd2 = pd_result[pd_result['model']=='2'].groupby(['model','tokenizer','embedding '])[pd_result.columns[3:]].sum().reset_index()
pd3 = pd_result[pd_result['model']=='3'].groupby(['model','tokenizer','embedding '])[pd_result.columns[3:]].sum().reset_index()
pd4 = pd_result[pd_result['model']=='4'].groupby(['model','tokenizer','embedding '])[pd_result.columns[3:]].sum().reset_index()
pd_result = pd.concat([pd1,pd2,pd3,pd4])
#pd_result
pd_result.to_csv('result.csv')

In [ ]:
pd_result = pd_result.set_index('model').sort_values(by=['DCG@1000'], ascending=False)

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Были построены четыре различные модели для решения задачи ранжирования твитов.

- 1 - модель, в которой использовались ембеддинги из предообученной модели Word2Vec от gensim 'SO_vectors_200.bin'. Для токенизации использовался простейший токенизатор.
- 2 - модель, в которой использовались ембеддинги из предообученной модели Word2Vec от gensim 'SO_vectors_200.bin'. Для токенизации использовался токенизатор nltk WordPunctTokenizer. Кроме того, убирались знаки пунктуации и стопслова
- 3 - модель, в которой использовались ембеддинги обученные для данной задачи. Для оценки контекта было выбрано окно шириной 5 токенов. Для токенизации использовался простейший токенизатор.
- 4 - модель, в которой использовались ембеддинги обученные для данной задачи. Для оценки контекта было выбрано окно шириной 5 токенов. Для токенизации использовался токенизатор nltk WordPunctTokenizer. Кроме того, убирались знаки пунктуации и стопслова

Полученные результаты сведены в итоговую таблицу:

In [ ]:
pd_result

,tokenizer,embedding,DCG@ 1,Hits@ 1,DCG@ 5,Hits@ 5,DCG@ 10,Hits@ 10,DCG@ 100,Hits@ 100,DCG@ 500,Hits@ 500,DCG@1000,Hits@1000
model,,,,,,,,,,,,,,
2,nltk + stopwords,word2vec SO_vectors_200.bin,0.395,0.395,0.474,0.545,0.496,0.614,0.544,0.844,0.560,0.969,0.564,1.0
4,nltk + stopwords,trained,0.374,0.374,0.472,0.555,0.494,0.624,0.542,0.859,0.557,0.970,0.560,1.0
1,simple,word2vec SO_vectors_200.bin,0.285,0.285,0.342,0.393,0.360,0.449,0.406,0.679,0.431,0.879,0.444,1.0
3,simple,trained,0.261,0.261,0.324,0.382,0.349,0.459,0.402,0.718,0.428,0.919,0.436,1.0


 - токенизатор с использованием стопслов и очисткой знаков пунктуации показал гораздо лучшие результаты при прочих равных условиях. Попарное сравнение моделей 1 и 2, 3 и 4.
 
 - лучшим образом с задачей справляются предобученные эмбеддинги word2vec SO_vectors_200.bin. Модель №2.
 
 - возможно качество решения задачи удастся улучшить, построив модели в которых учитывается порядок следования слов